In [1]:
"DOING THE SAME AS FOR THE MME_DATASET, BUT THEN FOR ONLY COUNTRIES *NOT* IN THE WESTERN LIST"

'DOING THE SAME AS FOR THE MME_DATASET, BUT THEN FOR ONLY COUNTRIES *NOT* IN THE WESTERN LIST'

In [ ]:
import pandas as pd

# create a dataframe with only the UserCountry3 column from the 71M SharedResponses.csv file
reader_country = pd.read_csv('SharedResponses.csv', usecols=['UserCountry3', 'ResponseID'])

In [4]:
# List of countries considered 'Western' from the UN

countries = ["AND",  "AUS", "AUT", "BEL", "CAN", "CHE", "DNK", "DEU", "ESP", "FIN",  "FRA",  "GBR",  "GRC",  "IRL", 
            "ISR", "ISL", "ITA", "LIE", "LUX", "MLT", "MCO", "NLD", "NOR", "NZL", "PRT", "SMR", "SWE", "TUR", "USA" ]

# Andorra, Australia, Austria, Belgium, Canada, Switzerland, Denmark, Germany, Spain, Finland, France, United Kingdom, Greece, Ireland, 
# Israel, Iceland, Italy, Liechtenstein, Luxembourg, Malta, Monaco, Netherlands, Norway, New Zealand, Portugal, San Marino, Sweden, Turkey, USA

In [5]:
# Filter the rows where UserCountryID is NOT the list of countries that are considered Western

non_western = reader_country[~reader_country['UserCountry3'].isin(countries)]

In [6]:
non_western.shape

(21837017, 2)

In [7]:
# only keep complete scenarios of 2

# Count how many times each ResponseID appears
response_counts = non_western['ResponseID'].value_counts()

# Get the ResponseIDs that appear exactly twice
ids_to_keep = response_counts[response_counts == 2].index

# Filter the rows where ResponseID is in the list of ids_to_keep
complete_responseid_country = non_western[non_western['ResponseID'].isin(ids_to_keep)]

In [8]:
print(complete_responseid_country.value_counts().min())
print(complete_responseid_country.value_counts().max())

2
2


In [9]:
complete_responseid_country.shape

(21084418, 2)

In [10]:
# checking the NAN's in UserCountry3 in the complete_responseid_country dataset and removing them
print(complete_responseid_country['UserCountry3'].isna().sum())

644868


In [11]:
# removing the NAN's
complete_responseid_country = complete_responseid_country.dropna(subset=['UserCountry3'])
print(complete_responseid_country['UserCountry3'].isna().sum())
print(complete_responseid_country.shape)

0
(20439550, 2)


In [12]:
# Drop duplicates based on 'ResponseID' and keep the first occurrence
complete_responseid_country = complete_responseid_country.drop_duplicates(subset=['ResponseID'], keep='first')

In [13]:
country_subset = complete_responseid_country.sample(n=2000000)  # want 4M rows (with accounting for NAn's and deleting rows with 'random' (around 10%)), so 4M / 2 = 2M ResponseID's necessary

In [14]:
# checking if they are all unique

country_subset.nunique()

ResponseID      2000000
UserCountry3        201
dtype: int64

In [15]:
# transform to a list to feed to the for loop that will extract all the corresponding columns from the SharedResponses.csv file

country_sub_list = country_subset['ResponseID'].tolist()
# check if it went okay
print(len(country_sub_list))     # should be 2000000

2000000


In [16]:
subset_country = pd.DataFrame()

In [17]:
# reading SharedResponses to extract the rows with ResponseID's in country_sub_list
# this will result in a dataframe 'subset' that contains around 17 million rows (8,5 * 2)

chunk_size = 500_000
reader = pd.read_csv('SharedResponses.csv', chunksize=chunk_size, dtype=str, low_memory=False)

for i, chunk in enumerate(reader):
    
    print(f"Processing chunk {i+1}")

    # Filter rows where ResponseID is in reader_subset
    subset_chunk = chunk[chunk['ResponseID'].isin(country_sub_list)]

    # Append filtered chunk to empty df
    subset_country = pd.concat([subset_country, subset_chunk], ignore_index=True)

    print(f"Finished processing chunk {i+1}")

print("All chunks have been processed and combined.")

Processing chunk 1
Finished processing chunk 1
Processing chunk 2
Finished processing chunk 2
Processing chunk 3
Finished processing chunk 3
Processing chunk 4
Finished processing chunk 4
Processing chunk 5
Finished processing chunk 5


KeyboardInterrupt: 

In [ ]:
subset_country.shape  # 4M rows, 41 columns

(4000000, 41)

In [ ]:
# checking to see if the UserCountry3 column only contains countries not in the Western list
print(subset_country['UserCountry3'].isin(countries).value_counts())

# should be False for all rows (4M)

UserCountry3
False    4000000
Name: count, dtype: int64


In [ ]:
subset_country_csv = subset_country.to_csv('subset_country.csv', index=False)

In [1]:
import pandas as pd
df_country = pd.read_csv('subset_country.csv')

In [2]:
df_country['ScenarioTypeStrict'].value_counts()

ScenarioTypeStrict
Utilitarian      717062
Species          707080
Age              705802
Fitness          705046
Gender           703620
Random           352396
Social Status    108994
Name: count, dtype: int64

In [3]:
# removing all rows with 'random' in the ScenarioTypeStrict column

df_country = df_country[df_country['ScenarioTypeStrict'] != 'Random']

In [4]:
df_country['ScenarioTypeStrict'].value_counts()

ScenarioTypeStrict
Utilitarian      717062
Species          707080
Age              705802
Fitness          705046
Gender           703620
Social Status    108994
Name: count, dtype: int64

In [5]:
# removing all rows with 'random' in the ScenarioTypeStrict column, as LLM's don't have a random scenario
df_country['AttributeLevel'].value_counts()

AttributeLevel
Less       358531
More       358531
Pets       353540
Hoomans    353540
Female     351810
Male       351810
Young      341067
Old        341067
Fat        318245
Fit        318245
Rand       116044
High        42587
Low         42587
Name: count, dtype: int64

In [6]:
df_country = df_country[df_country['AttributeLevel'] != 'Rand']

In [7]:
# removing all rows with 'Rand' in the AttributeLevel column, as LLM's don't have a random scenario
df_country['AttributeLevel'].value_counts()

AttributeLevel
Less       358531
More       358531
Pets       353540
Hoomans    353540
Female     351810
Male       351810
Young      341067
Old        341067
Fat        318245
Fit        318245
High        42587
Low         42587
Name: count, dtype: int64

In [8]:
df_country.shape

(3531560, 41)

In [9]:
df_country.isna().sum()

ResponseID                      0
ExtendedSessionID               0
UserID                        194
ScenarioOrder                   0
Intervention                    0
PedPed                          0
Barrier                         0
CrossingSignal                  0
AttributeLevel                  0
ScenarioTypeStrict              0
ScenarioType                    0
DefaultChoice                   0
NonDefaultChoice                0
DefaultChoiceIsOmission         0
NumberOfCharacters              0
DiffNumberOFCharacters          0
Saved                           0
Template                   421936
DescriptionShown           421936
LeftHand                   421936
UserCountry3                    0
Man                             0
Woman                           0
Pregnant                        0
Stroller                        0
OldMan                          0
OldWoman                        0
Boy                             0
Girl                            0
Homeless      

In [10]:
# deleting rows with NaN's in the UserID column

df_country = df_country.dropna(subset=['UserID'])

In [11]:
df_country.shape # should be 3531560 - 194 = 3531366

(3531366, 41)

In [32]:
# taking subset

In [ ]:
# the total dataset has to be 500000 rows.
# 50% of that is LLM's, so 250000 rows
# the other 50% will be humans, so 250000 rows

# need to subset 250000 rows from the 3531366 rows
# need to delete 3531366 - 250000 = 3281366 rows

In [12]:
# randomly delete 3281366 / 2 = 1640683 unique UserIDs (is 3281366 rows), to ensure 50% of the dataset is LLMs and 50% humans

# Getting unique UserIDs
Response_unique = df_country['ResponseID'].unique()
print(len(Response_unique))  # should be 3531366/2 = 1765683

# Selecting 1140683 UserIDs from the unique set
Response_delete = pd.Series(Response_unique).sample(n=1640683, random_state=42)


1765683


In [14]:
df_country_50 = df_country[~df_country['ResponseID'].isin(Response_delete)]

In [15]:
# checking shape of df_filtered

df_country_50.shape   # should be 1.25M rows

(250000, 41)

In [16]:
df_country_50.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
32,224cxgPeYZTakSxMj,803343358_4187351052140056.0,4.187351e+15,10,0,0,0,2,Fat,Fitness,...,0,0,0,0,0,0,0,0,0,0
44,225g5gXAySJ8wYvuu,691679560_112812281,1.128123e+08,10,0,1,0,0,Fit,Fitness,...,0,0,0,0,1,1,0,0,0,0
69,2289okSZKQ6PXH3iL,579690810_331595228388075.0,3.315952e+14,12,0,0,1,0,Old,Age,...,0,0,0,0,0,0,0,0,0,0
70,228ApeJeBM4tZFXqz,982385140_3818736516.0,3.818737e+09,7,0,0,0,2,Female,Gender,...,0,0,0,0,0,0,1,0,0,0
74,228PLGxziJnJomipC,-566646533_671024759001037.0,6.710248e+14,12,0,0,1,0,Hoomans,Species,...,0,0,0,0,0,1,0,0,0,0


In [18]:
# check what the distribution of RepsonseID's is (kinda)
# check if the indices of a responseID here are the same after transforming the ResponseID column
# if the indices are the same means ResponseID column is succesfully transformed

indices = df_country_50[df_country_50["ResponseID"] == '224cxgPeYZTakSxMj'].index
print(indices)

Index([32, 504674], dtype='int64')


In [20]:
# Changing the responseID

# Define the starting point for the new ResponseID
starting_id = 773566

# Step 1: Get the unique ResponseIDs
unique_response_ids = df_country_50['ResponseID'].unique()

# Step 2: Create a mapping from old ResponseID to new 'res_' formatted ID
response_id_mapping = {old_id: f'res_{i:08d}' for i, old_id in enumerate(unique_response_ids, starting_id)}

# Step 3: Replace the original ResponseID with the new mapped IDs
df_country_50['ResponseID'] = df_country_50['ResponseID'].map(response_id_mapping)

C:\Users\esmku\AppData\Local\Temp\ipykernel_32248\1410160540.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country_50['ResponseID'] = df_country_50['ResponseID'].map(response_id_mapping)


In [21]:
# ResponseID starts with res_00146784
df_country_50.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
32,res_00773566,803343358_4187351052140056.0,4.187351e+15,10,0,0,0,2,Fat,Fitness,...,0,0,0,0,0,0,0,0,0,0
44,res_00773567,691679560_112812281,1.128123e+08,10,0,1,0,0,Fit,Fitness,...,0,0,0,0,1,1,0,0,0,0
69,res_00773568,579690810_331595228388075.0,3.315952e+14,12,0,0,1,0,Old,Age,...,0,0,0,0,0,0,0,0,0,0
70,res_00773569,982385140_3818736516.0,3.818737e+09,7,0,0,0,2,Female,Gender,...,0,0,0,0,0,0,1,0,0,0
74,res_00773570,-566646533_671024759001037.0,6.710248e+14,12,0,0,1,0,Hoomans,Species,...,0,0,0,0,0,1,0,0,0,0


In [22]:
indices = df_country_50[df_country_50["ResponseID"] == 'res_00773566'].index
print(indices)

# checking if the indices are the same after transforming the ResponseID column
# compared to the indices of the ResponseID before transforming the column (which was 2224kBG72574tbZD3) - they are!
# both are [2, 884580]

Index([32, 504674], dtype='int64')


In [23]:
df_country_50.tail()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
3999936,res_00898561,-2104339801_3965911106.0,3.965911e+09,1,1,0,0,0,Less,Utilitarian,...,1,0,0,0,0,0,3,0,0,0
3999942,res_00898562,-725957820_44932425,4.493242e+07,8,1,0,0,1,Less,Utilitarian,...,1,0,0,0,0,0,0,0,0,1
3999968,res_00898563,773763213_845246661,8.452467e+08,13,1,0,0,0,Low,Social Status,...,0,0,0,0,0,0,0,0,0,0
3999992,res_00898564,2096433344_3994632511834006.0,3.994633e+15,8,1,1,0,2,Male,Gender,...,0,0,0,0,0,0,0,0,0,0
3999994,res_00898565,636847514_7747179816721910.0,7.747180e+15,10,1,0,1,0,Female,Gender,...,0,0,0,0,2,0,1,0,0,0


In [24]:
df_country_50.columns

Index(['ResponseID', 'ExtendedSessionID', 'UserID', 'ScenarioOrder',
       'Intervention', 'PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel',
       'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice',
       'NonDefaultChoice', 'DefaultChoiceIsOmission', 'NumberOfCharacters',
       'DiffNumberOFCharacters', 'Saved', 'Template', 'DescriptionShown',
       'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
       'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
       'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
       'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
       'Cat'],
      dtype='object')

In [25]:
# already deleting some columns to ensure it fits in memory
# deleting the columns that are not necessary for the modelling: ExtendedSessionID, DefaultChoice, NonDefaultChoice, DefaultChoiceIsOmission, Template

df_country_clean = df_country_50.drop(columns=['ExtendedSessionID', 'DefaultChoice', 'NonDefaultChoice', 'DefaultChoiceIsOmission', 'Template', 'ScenarioType', 'ScenarioOrder', 'DescriptionShown', 'LeftHand', 'UserCountry3'])

In [26]:
df_country_clean.shape # should be 31 columns

(250000, 31)

In [27]:
# resetting the index
df_country_clean = df_country_clean.reset_index(drop=True)

In [28]:
# binarizing UserID - making all into 0
df_country_clean['UserID'] = 0

In [29]:
# saving this pre-preprocessed non-western dataset to a csv file

df_country_clean.to_csv('nw_50_dataset.csv', index=False)